In [13]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import sqlalchemy as sf
import psycopg2

In [14]:
from sql_functions import get_engine
from sql_functions import get_dataframe
from sql_functions import get_sql_config

# Step 1: Import file and create dataframe with individual variable

In [15]:

smartprotein_consumer = pd.read_excel("data/Ernährungsweisen_data/Smart Protein - EU consumer survey.xlsx")

In [16]:
smartprotein_consumer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   q_no             113 non-null    object 
 1   question         113 non-null    object 
 2   answer           113 non-null    object 
 3   Total            113 non-null    float64
 4   Austria          113 non-null    float64
 5   Denmark          113 non-null    float64
 6   France           113 non-null    float64
 7   Germany          113 non-null    float64
 8   Italy            113 non-null    float64
 9   The Netherlands  113 non-null    float64
 10  Poland           113 non-null    float64
 11  Romania          113 non-null    float64
 12  Spain            113 non-null    float64
 13  UK               113 non-null    float64
dtypes: float64(11), object(3)
memory usage: 12.5+ KB


# Step 4: Rename columns lower case, snake case, spaces, delimiters

In [17]:
smartprotein_consumer.head()

,q_no,question,answer,Total,Austria,Denmark,France,Germany,Italy,The Netherlands,Poland,Romania,Spain,UK
0,Q1,Which category best describes your current die...,Omnivore,61.0,56.0,62.0,65.0,58.0,67.0,48.0,68.0,54.0,62.0,65.0
1,Q1,Which category best describes your current die...,Flexitarian,30.0,35.0,24.0,27.0,30.0,25.0,42.0,24.0,40.0,30.0,23.0
2,Q1,Which category best describes your current die...,Pescetarian,3.0,2.0,5.0,4.0,3.0,3.0,4.0,2.0,3.0,2.0,3.0
3,Q1,Which category best describes your current die...,Vegetarian,5.0,5.0,5.0,4.0,7.0,5.0,5.0,5.0,3.0,3.0,6.0
4,Q1,Which category best describes your current die...,Vegan,2.0,2.0,4.0,2.0,3.0,1.0,2.0,1.0,1.0,3.0,3.0


In [6]:
#smartprotein_consumer.columns = smartprotein_consumer.columns.str.lower()
smartprotein_consumer.columns = smartprotein_consumer.columns.str.replace('The ','')
smartprotein_consumer.columns = smartprotein_consumer.columns.str.replace('UK','United Kingdom')
smartprotein_consumer.columns = smartprotein_consumer.columns.str.strip()

In [7]:
smartprotein_consumer.columns

Index(['q_no', 'question', 'answer', 'Total', 'Austria', 'Denmark', 'France',
       'Germany', 'Italy', 'Netherlands', 'Poland', 'Romania', 'Spain',
       'United Kingdom'],
      dtype='object')

In [8]:
smartprotein_consumer = smartprotein_consumer.melt(id_vars=["q_no", "question", "answer"], # Spalten die gleich bleiben sollen
                                                           var_name="country", # neue column
                                                         value_name="no_%") # welche Werte sollen zugeordnet werden

In [9]:
smartprotein_consumer.tail(20)

,q_no,question,answer,country,no_%
1223,Q24,Which of the following would you like to have ...,Potatoes,United Kingdom,37.0
1224,Q24,Which of the following would you like to have ...,Rice,United Kingdom,34.0
1225,Q24,Which of the following would you like to have ...,Almond,United Kingdom,26.0
1226,Q24,Which of the following would you like to have ...,Corn,United Kingdom,25.0
1227,Q24,Which of the following would you like to have ...,Beans,United Kingdom,29.0
1228,Q24,Which of the following would you like to have ...,Mushroom,United Kingdom,31.0
1229,Q24,Which of the following would you like to have ...,Lentils,United Kingdom,26.0
1230,Q24,Which of the following would you like to have ...,Peas,United Kingdom,26.0
1231,Q24,Which of the following would you like to have ...,Hazelnuts,United Kingdom,22.0
1232,Q24,Which of the following would you like to have ...,Chickpeas,United Kingdom,24.0


In [10]:
smartprotein_consumer.country.unique()

array(['Total', 'Austria', 'Denmark', 'France', 'Germany', 'Italy',
       'Netherlands', 'Poland', 'Romania', 'Spain', 'United Kingdom'],
      dtype=object)

In [11]:
##smartprotein_consumer

# Import get_engine from sql_functions.py. You will need to restart your kernel and rerun at this point since we changed the module since we first imported it.
from sql_functions import get_engine
# create a variable called engine using the get_engine function
engine = get_engine()

# Set the schema to your capstone schema and the table_name variable to 'fish_catch' etc + your initials/group number

schema = 'capstone_fish_are_friends' # capstone schema example capstone_fish_are_friends
table_name = 'smartprotein_consumer' # Example: 'carriers_pw' for Philipp Wendt


# Write records stored in a dataframe to SQL database using to_sql() function
if engine!=None:
   try:
      smartprotein_consumer.to_sql(name=table_name, # Name of SQL table variable
                        con=get_engine(), # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
      print(f"The {table_name} table was imported successfully.")
    # Error handling
   except (Exception, psycopg2.DatabaseError) as error:
      print(error)
      engine = None
else:
   print('No engine')

(psycopg2.OperationalError) could not translate host name "host" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/14/e3q8)
